In [ ]:
# smooth data

def smoothed(x, n=300, k=3, y=None, return_type='df', datetime_index=False):
    '''
    Smooth data for plots
    
    Arguments:
    x: pd.DataFrame, pd.Series
    y: array-type
    n: length of linespace
    k: smoothing scale
    return_type: 
        - if 'array' - return x_new, y_new
        - if 'dict' - returns dict with {'x': x_new, 'y': y_new}

    If x == pd.DataFrame functon returns pd.DataFrame anyway

    Libraries:
    from scipy.interpolate import make_interp_spline, BSpline
    '''

    if datetime_index:
        start = x.index[0]
        end = x.index[-1]
        time_range = \
            pd.date_range(start=start, end=end, periods=n)
        x = x.reset_index(drop=True)

    if isinstance(x, pd.DataFrame):
        var_name = x.columns[0] if x.columns[0] != 0 else 'variable'
        x_index = x.index
        x_new = np.linspace(x_index.min(), x_index.max(), n)
        df = pd.DataFrame(index=x_new, columns=x.columns)
        for col in x.columns:
            y = x[col]
            spl = scipy.interpolate.make_interp_spline(x_index, y, k=k)  # type: BSpline
            y_new = spl(x_new)
            df[col] = y_new
        if return_type == 'df':
            if datetime_index:
                df.index = time_range
            return df
        if return_type == 'array':
            return np.array(df.index), np.array(df.iloc[:, 0])
        
    elif isinstance(x, pd.Series):
        var_name = x.name
        y = x.copy()
        x = x.index
        
        # n represents number of points to make between T.min and T.max
        x_new = np.linspace(x.min(), x.max(), n) 
    
        spl = scipy.interpolate.make_interp_spline(x, y, k=k)  # type: BSpline
        y_new = spl(x_new)
    
        if return_type == 'dict':
            if datetime_index:
                ret_dict = {
                    'x': time_range,
                    'y': y_new
                    }
            else:
                ret_dict = {
                    'x': x_new,
                    'y': y_new
                    }
            return ret_dict
        elif return_type == 'array':
            if datetime_index:
                return time_range, y_new
            else:
                return x_new, y_new
        elif return_type == 'df':
            if datetime_index:
                df = pd.DataFrame(data=y_new, index=time_range, columns=[var_name])
            else:
                df = pd.DataFrame(data=y_new, index=x_new, columns=[var_name])
            return df
    else:
        y = x.copy()
        x = arange(len(x))

        # n represents number of points to make between T.min and T.max
        x_new = np.linspace(x.min(), x.max(), n) 
    
        spl = scipy.interpolate.make_interp_spline(x, y, k=k)  # type: BSpline
        y_new = spl(x_new)
        
        if return_type == 'dict':
            if datetime_index:
                ret_dict = {
                    'x': time_range,
                    'y': y_new
                    }
            else:
                ret_dict = {
                    'x': x_new,
                    'y': y_new
                    }
            return ret_dict
        elif return_type == 'array':
            if datetime_index:
                return time_range, y_new
            else:
                return x_new, y_new
        elif return_type == 'df':
            if datetime_index:
                df = pd.DataFrame(data=y_new, index=time_range, columns=['variable'])
            else:
                df = pd.DataFrame(data=y_new, index=x_new, columns=['variable'])
            return df

In [ ]:
# check if there are NaNs in df

def is_nan(df):
    ret = df[df.isna().any(axis=1)]
    shape = df[df.isna().any(axis=1)].shape
    if shape[0] > 0:
        return ret
    else:
        print("No NaN values in DataFrame")

In [ ]:
def data_describe(data):
    
    df = data.copy()
    # varibles types
    dtypes = df.dtypes.rename('Type').to_frame()
    # frequency
    frequency = df.count().rename('Count').to_frame()
    # unique values
    unique = df.nunique().rename('Unique').to_frame()
    # NaNs
    nans = df.isnull().sum().rename('NaN').to_frame()
    # NaNs fraction
    nans_frac = df.isnull().mean().round(2)
    nans_frac = nans_frac.rename('Percentages').to_frame()
    # list with results
    results_list = [dtypes, frequency, unique, nans, nans_frac]
    # df with results
    results = pd.concat(results_list, axis=1)
    results['Percentages'] = (results['Percentages'] * 100).astype('int64')
    results = results.sort_values(['NaN'], ascending=False)
    
    return results

In [ ]:
def ci_bootstrap(
        data, statistic=np.mean, n_bootstrap=1000,
        confidence_level=0.95, random_state=42):
    '''
    Returns: dict(statistic, std, ci_min, ci_max, margin)
    '''
    data_ = (data,)
    bootstrap = scipy.stats.bootstrap(
        data=data_,
        statistic=statistic,
        n_resamples=n_bootstrap,
        confidence_level=confidence_level,
        random_state=random_state
    )
    ci_min = bootstrap.confidence_interval[0]
    ci_max = bootstrap.confidence_interval[1]
    if isinstance(data, pd.DataFrame):
        stat = data.apply(statistic)
        stat = np.array(stat)
        std = np.array(np.std(data, ddof=1))
    else:
        stat = statistic(data)
        std = np.std(data, ddof=1)
    margin = stat - ci_min

    return_dct = {
        'statistic': stat,
        'std': std,
        'ci_min': ci_min,
        'ci_max': ci_max,
        'margin': margin,
    }
    return return_dct

In [ ]:
def ci_t_distribution(
        data=None, mean=None, std=None, n=None, confidence_level=0.95):

    if data is not None:
        arr = np.array(data)
        n = len(arr)
        mean = np.mean(arr)
        se = scipy.stats.sem(arr)
        
    if mean and std and n is not None:
        se = std / np.sqrt(n)

    t = scipy.stats.t.ppf((1+confidence_level) / 2, n-1)
    margin = t * se
    ci_min = mean - margin
    ci_max = mean + margin

    return_dct = {
        'mean': mean,
        'ci_min': ci_min,
        'ci_max': ci_max,
        'margin': margin,
        't': t
    }
    return return_dct

In [ ]:
# normality tests

def test_normality(data, alpha=0.05):
    
    tests_names = []
    pvalue = []
    condition = []
        
    # Kolmogorov-Smirnov
    ks = stats.kstest(data, 'norm')
    pvalue_ks = ks.pvalue
    tests_names.append('Kolmogorov-Smirnov')
    pvalue.append(pvalue_ks)
    if pvalue_ks < alpha:
        condition.append('Not normal')
    else:
        condition.append('Normal')

    # Anderson-Darling
    and_dar = stats.anderson(data, dist='norm')
    and_dar_sign = and_dar.critical_values[2]
    and_dar_statistic = and_dar.statistic
    tests_names.append('Anderson-Darling (s)')
    pvalue.append(and_dar_statistic)
    if and_dar_statistic > and_dar_sign:
        condition.append('Not normal')
    else:
        condition.append('Normal')

    # Shapiro-Wilk
    pvalue_sw = stats.shapiro(data).pvalue
    tests_names.append('Shapiro-Wilk')
    pvalue.append(pvalue_sw)
    if pvalue_sw < alpha:
        condition.append('Not normal')
    else:
        condition.append('Normal')

    # jarque-bera test
    jb_name = ["Jarque-Bera", "Chi^2", "Skew", "Kurtosis"]
    jb_statistic = sms.jarque_bera(data)
    jb = dict(zip(jb_name, jb_statistic))
    pvalue_jb = jb['Chi^2']
    tests_names.append('Jarque-Bera')
    pvalue.append(pvalue_jb)
    if pvalue_jb < alpha:
        condition.append('Not normal')
    else:
        condition.append('Normal')
    
    # D’Agostino and Pearson
    dagp = stats.normaltest(data)
    pvalue_dagp = dagp.pvalue
    tests_names.append('D’Agostino-Pearson')
    pvalue.append(pvalue_dagp)
    if pvalue_dagp < alpha:
        condition.append('Not normal')
    else:
        condition.append('Normal')

    pvalue = [np.round(i, 4) for i in pvalue]
    results_df = pd.DataFrame({
        'Test': tests_names,
        'P or Statistic (s)': pvalue,
        'Condition': condition,
    })
    
    return results_df

In [ ]:
def feature_importance_display(
        features, importance,
        top=None, imp_min_level=None, only_features=True):

    '''
     
    '''

    feature_importance = pd.DataFrame({
        'Feature': features,
        'Importance': importance
    })
    if imp_min_level is not None:
        loc_row = feature_importance['Importance'] > imp_min_level
        feature_importance = (feature_importance
                              .loc[loc_row, :]
                              .sort_values('Importance', ascending=False)
                              .reset_index(drop=True))
    if top is not None:
        feature_importance = (feature_importance
                             .sort_values('Importance', ascending=False)
                             .reset_index(drop=True))
        feature_importance = feature_importance.loc[0:top-1]

    if only_features:
        feature_importance = feature_importance['Feature']
        
    return feature_importance

In [ ]:
def outliers_column_iqr(data, feature, scale=1.5):

    '''
    Add nominative (1/0) column '{feature}_is_out' in DataFrame, that indicates outliers for Feature
    '''

    df = data.copy()

    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1
    lower_boundary = q1 - scale*iqr
    upper_boundary = q3 + scale*iqr
    condition = ((df[feature] < lower_boundary) |
                 (df[feature] > upper_boundary))
    df[feature+'_is_out'] = condition.astype(int)

    return df

In [ ]:
def correlation_w_target(data, target):

    '''
    Create sorted DataFrame with correlations to Target 
    '''
    
    df = (data
          .corr()[target]
          .sort_values(ascending=False, key=abs)[1:]
          .to_frame())
    return df

In [ ]:
def check_columns_match(data):

    '''
    Check if all columns in DataFrame are equal and return no equal if not
    '''

    df = data.copy()
    df['is_equal'] = df.eq(df.iloc[:, 0], axis=0).all(1).astype(int)
    equal_sum = df['is_equal'].sum()

    if equal_sum == len(df):
        print('All values matched')
        return None
    else:
        loc = df['is_equal'] == 0, df.columns != 'is_equal'
        result = df.loc[loc].copy()
        return result      

In [ ]:
def fillna_na(data, features_list):

    '''
    Fill all NaNs in DataFrame by 'NA'
    '''

    df = data.copy()
    for feature in features_list:
        df[feature] = df[feature].fillna('NA')

    return df

In [ ]:
def normalized_by_first(data, return_type='df'):

    '''
    Normalize kind: 
        first_value == first_value
        second_value = second_value / first_value
        third_value = third_value / first_value
    '''
    
    first_value = data[0]
    
    data_new = [(x/first_value) for x in data]

    if return_type == 'df':
        df = pd.DataFrame(data=data_new, index=data.index)
        return df
    if return_type == 'series':
        series = pd.Series(data=data_new, index=data.index)
        return series
    elif return_type == 'array':
        array = np.array(data_new)
        return array
    elif return_type == 'list':
        lst = list(data_new)
        return lst
    else:
        print("'return_type' must be 'df', 'series', 'array', 'list'")
    
    return data_new

In [ ]:
def normalized(data, reshape=True, return_type='df'):

    '''
    MinMaxScaler 0/1 
    '''
    
    if (isinstance(data, pd.Series) | 
        isinstance(data, pd.DataFrame)):
        idxs = data.index.copy()
    if reshape:
        data = np.array(data).reshape(-1, 1)
    data_new = MinMaxScaler().fit_transform(data)
    if return_type == 'df':
        data_new = pd.DataFrame(data=data_new, index=idxs)
    elif return_type == 'array':
        pass
    else:
        print("return_type must be 'df' or 'array'")
        return None
        
    return data_new

In [ ]:
def skewness(df):

    df = pd.DataFrame(df.skew(numeric_only=True),
                      columns=['Skewness'],
                      index=None)

    df['Highly skewed'] = (abs(df['Skewness']) > 0.5)
    df['abs'] = abs(df['Skewness'])

    df = df.sort_values(by=['abs', 'Highly skewed'], ascending=False)
    df = df.drop('abs', axis=1)

    return df

In [ ]:
def kurtosis(df):

    df = pd.DataFrame(df.kurtosis(numeric_only=True),
                      columns=['Kurtosis'],
                      index=None)
    df['Type'] = np.nan

    df.loc[df['Kurtosis'] > 1, 'Type'] = 'Too Peaked'
    df.loc[df['Kurtosis'] < -1, 'Type'] = 'Too Flat'
    df.loc[(df['Kurtosis'] <= 1) & (df['Kurtosis'] >= -1), 'Type'] = 'Normal'
    
    df['abs'] = abs(df['Kurtosis'])
    df = df.sort_values(by=['abs', 'Type'], ascending=False)
    df = df.drop('abs', axis=1)

    return df

In [ ]:
def plot_acf(
        acf_w_alphas=None, data=None, lags=40, partial=False, scatter=False, s=2,
        transparency_lines=1, color_lines=None, exclude_first=True,
        transparency_significant=0.15, show_last_significant=True,
        last_significant_delta=0.1, color_significant=None, **kwargs):

    if acf_w_alphas is None:
        acf_w_alphas = ts_acf_calculate(data, lags=lags, partial=partial, **kwargs) 
        
    acf = acf_w_alphas[:, 0]
    alphas = acf_w_alphas[:, 1:]
    
    lags = len(acf)
    xticks = arange(lags)
    color_palette = plt.rcParams['axes.prop_cycle'].by_key()['color']

    color_significant = color_significant or color_palette[2]
    color_lines = color_lines or color_palette[0]

    if exclude_first:
        acf[0] = 0
        alphas[:1] = 0

    if scatter:
        plt.scatter(
            x=xticks,
            y=acf,
            s=s
        )
    for i in arange(lags):
        plt.plot(
            [i, i],
            [0, acf[i]],
            color=color_lines,
            alpha=transparency_lines
        )
    if exclude_first:
        plt.fill_between(
            arange(lags)[1:],
            (alphas[:, 0] - acf)[1:],
            (alphas[:, 1] - acf)[1:],
            lw=0,
            color=color_significant,
            alpha=transparency_significant
        )
    else:
        plt.fill_between(
            arange(lags),
            alphas[:, 0] - acf,
            alphas[:, 1] - acf,
            lw=0,
            color=color_significant,
            alpha=transparency_significant
        )

    if show_last_significant:
        last_sign = ts_acf_last_significant_index(data=data, partial=partial)
        pacf_text = f'{last_sign}'
        last_sign_y = acf[last_sign] + last_significant_delta
    
        plt.annotate(
            text=pacf_text,
            xy=(last_sign, last_sign_y),
            ha='center',
            size=9,
            color=palette[1],
            weight='bold')

    plt.plot([-1, lags], [0, 0])
    plt.gca().spines[['bottom', 'left']].set_visible(False)
    plt.grid(False)
    plt.xlim(-2, lags+1)
    # plt.show()

In [ ]:
def ts_acf_calculate(data, lags=36, alpha=0.05, partial=False, **kwargs):

    if partial:
        acf_result = statsmodels.tsa.stattools.pacf(
            data, nlags=lags, alpha=alpha, method='ywadjusted', **kwargs)
    else:
        acf_result = statsmodels.tsa.stattools.acf(
            data, nlags=lags, alpha=alpha, missing='none', **kwargs)

    acf = acf_result[0]
    alphas = acf_result[1]
    result = np.hstack([acf.reshape(-1,1), alphas])
    
    return result

In [ ]:
def ts_acf_last_significant_index(data, lags=36, partial=False):
    '''
    Return index of first insignificant element in ACF or PACF

    Attributes:
        ci - confident intervals for ACF value (example, result[1] of statsmodels.tsa.stattools.acf)
    '''
    acf = ts_acf_calculate(data, lags=lags, partial=partial)
    ci = acf[:, 1:]
    
    # for i, j in enumerate(ci):
    #     status = np.all(j > 0) if j[0] > 0 else np.all(j < 0)
    #     if not status:
    #         break
    for i, j in enumerate(ci):
        # check if values in 'alphas' have not equal sign
        if ((j[0]<0) != (j[1]<0)):
            return i-1
        elif i == len(ci)-1:
            print(f'All {lags} lags significant')

In [ ]:
def ts_arima_forecast(model, steps, data, ci=[80, 95]):

    df = data.copy()
    results = model.get_forecast(steps=steps)

    final_df = pd.DataFrame(
        index = pd.date_range(
            df.index[0], results.predicted_mean.index[-1], freq=df.index.freq),
        data=pd.concat([
            df.iloc[:,0], results.predicted_mean], axis=0),
        columns=['data'])

    
    final_df['forecast'] = np.where(
        final_df.index.date < results.predicted_mean.index[0].date(), 0, 1)

    for ci_value in ci:
        alpha = (100 - ci_value) / 100
        final_df[f'lower_ci{ci_value}'] = \
            results.conf_int(alpha=alpha).iloc[:, 0]
        final_df[f'upper_ci{ci_value}'] = \
            results.conf_int(alpha=alpha).iloc[:, 1]

    return final_df

In [ ]:
def test_poisson_bootstrap(
        data1, 
        data2,
        n_bootstrap=10000,
        ci=[2.5,97.5],
        rnd = 3,
        plot=True,
        figsize=(7, 2),
        colors=None,
        execution_time=True,
        results_dict=False,
        means_plots=True,
        rstyle=True,
        rstyle_dataplot_kwargs={},
        rstyle_meansplot_kwargs={},
        simple_results=False):

    '''
    If plot == True and results_dict == True and means_plots == True
    Returns: 
        - dict with means difference value and boundaries
        - dict with Poisson bootstrap folds
        - figure with means diffrenece plot with boundaries
        - figure with data plots
    '''

    if simple_results:
        plot = False
        execution_time = False
        results_dict = False
        means_plots = False
    
    t_start = time.time()

    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    color0 = colors[0]
    color1 = colors[1]
    color2 = '#505050'

    if not isinstance(data1, np.ndarray):
        data1 = np.array(data1)
    if not isinstance(data2, np.ndarray):
        data2 = np.array(data2)
    
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)

    means_diff = mean1 - mean2

    poisson_bootstraps1 = stats.poisson(1).rvs(
        (n_bootstrap, len(data1))).astype(np.int64)

    poisson_bootstraps2 = stats.poisson(1).rvs(
        (n_bootstrap, len(data2))).astype(np.int64)

    mean1_boot = (poisson_bootstraps1*data1).sum(axis=1) / len(data1)
    mean2_boot = (poisson_bootstraps2*data2).sum(axis=1) / len(data2)
    means_diff_boot = mean1_boot - mean2_boot
    
    lower_boundary, upper_boundary = np.percentile(means_diff_boot, ci)
    
    means_dict = {
        'mean1': mean1_boot,
        'mean2': mean2_boot,
        'means_diff': means_diff_boot
    }
    
    results = {
        'Lower Boundary': lower_boundary, 
        'Means Difference': means_diff, 
        'Upper Boundary': upper_boundary
    }

    if not simple_results:
        print('\n'+'         Poisson bootstrap summary')

    if plot:

        fig_data = plt.figure(figsize=figsize)

        ax = sns.histplot(
            means_diff_boot,
            color=color0, alpha=0.5)

        ylim = ax.get_ylim()[1]
        
        ax.vlines(
            0, 0, ylim*0.1,
            color=saturate_color(color1, 1.25), linewidth=2.5)
        ax.vlines(
            lower_boundary, 0, ylim*0.15,
            color=color2, linewidth=1.5)
        ax.vlines(
            upper_boundary, 0, ylim*0.15,
            color=color2, linewidth=1.5) 

        ax.set_ylabel('Count')

        if rstyle:
            axis_rstyle(**rstyle_dataplot_kwargs)

        ax.legend(
            **legend_inline(),
            **legend_create_handles(
                3, ['r', 'l', 'l'],
                colors=[color0, color1, color2],
                alphas=[0.5, 1, 1],
                labels=['Means difference', 'Zero', 'Significance borders'],
                linelength=1
            ))
        
        plt.show()
        
    # the boundaries, measured by 1 and 99 percentiles,
    # are equvivalent of p-value probabiblities boundaries an 0.05 significant level;
    # if difference in means is out of boundaries range, we reject null hypotesis - 
    # it means that the difference if statistical significant
    if lower_boundary < 0 < upper_boundary:
        significancy = False
    else: 
        significancy = True
    
    # check with Kolmogorov–Smirnov test if distribution of p-values is normal
    # (previously standardize means differences with stats.zscore)
    pvalue_ks = stats.kstest(stats.zscore(means_diff_boot), stats.norm.cdf).pvalue
    
    # Kolmogorov–Smirnov test null hypotesis: distribution of simulation pvalues is normal
    # if pvalue due Kolmogorov–Smirnov test <= 0.05, 
    # we reject null hypotesis that distribution of pvalues due simulation is normal;  
    if pvalue_ks <= 0.05:
        distribution = 'not '
    else:
        distribution = ''

    if not simple_results:

        mean1_rnd = f"%.{rnd}f" % mean1
        mean2_rnd = f"%.{rnd}f" % mean2
        lower_boundary_rnd = f"%.{rnd}f" % lower_boundary
        means_diff_rnd = f"%.{rnd}f" % means_diff
        upper_boundary_rnd = f"%.{rnd}f" % upper_boundary
        pvalue_rnd = f"%.{rnd}f" % pvalue_ks
        
        ha1 = '===================================================================================='
        start1 = '          '
        space = '              '

        delta1 = 27 - len(f'Lower Boundary:{lower_boundary_rnd}')
        delta1 = delta1 * ' '
        delta2 = 27 - len(f'Means Difference:{means_diff_rnd}')
        delta2 = delta2 * ' '
        delta3 = 27 - len(f'Upper Boundary:{upper_boundary_rnd}')
        delta3 = delta3 * ' '

        delta4 = 43 - len(f'Significantly difference:{significancy}')
        delta4 = delta4 * ' '
        delta5 = 43 - len(f"Means Differences' distribution:{distribution}normal")
        delta5 = delta5 * ' '
        delta6 = 43 - len(f'Kolmogorov–Smirnov test p-value:{pvalue_rnd}')
        delta6 = delta6 * ' '

        print(
            '\n'
            f'{start1}'f'Significantly difference:{delta4}\033[1m{significancy}\033[0m' \
                + space + f'Lower Boundary:{delta1}{lower_boundary_rnd}' '\n' 
            f'{start1}'f"Means Differences' distribution:{delta5}{distribution}normal" \
                + space + f'Means Difference:{delta2}{means_diff_rnd}' '\n' 
            f'{start1}'+f'Kolmogorov–Smirnov test p-value:{delta6}{pvalue_rnd}' \
                + space+ f'Upper Boundary:{delta3}{upper_boundary_rnd}' '\n' '\n' 
            f'{start1}' + ha1 + '\n' '\n' \
            f'{start1}' + f'Mean1: {mean1_rnd}' '\n'
            f'{start1}' + f'Mean2: {mean2_rnd}' '\n'
           )

    if means_plots:
        
        fig_means = plt.figure(figsize=figsize)
        
        ax = sns.histplot(
            mean1_boot,
            color=color0, alpha=0.5)
        
        ax = sns.histplot(
            mean2_boot, 
            color=color1, alpha=0.5)
        
        ax.set(xlabel=None)
        ax.set_ylabel('Count', weight='bold')
        
        ylim = ax.get_ylim()[1]
        
        ax.vlines(
            np.mean(mean1_boot), 0, ylim,
            color=saturate_color(color0, 1.25), linewidth=1, ls='--')
        ax.vlines(
            np.mean(mean2_boot), 0, ylim,
            color=saturate_color(color1, 1.25), linewidth=0.75, ls='--')

        if rstyle:
            axis_rstyle(**rstyle_meansplot_kwargs)

        ax.legend(
            **legend_inline(),
            **legend_create_handles(
                2, 's',
                colors=[color0, color1],
                alphas=[0.5, 0.5],
                labels=['Data1 means', 'Data2 means']
            ))
        
        plt.show()

    if execution_time:
        
        execution_time = np.round(time.time() - t_start, 2)
        execution_time_formated = \
                         str(dt.timedelta(seconds=np.round(time.time() - t_start)))
        
        print('\n'+f'{start1}'+'Execution time: {}'.format(execution_time_formated))
        print(f'{start1}'+'Execution time (seconds): {}'.format(execution_time, '\n'))

    if results_dict:
        return results, means_dict, fig_data, fig_means

    if simple_results:
        return significancy

In [232]:
def generate_feature_diff_sunday(data, feature):

    df = data.copy()
    df['month'] = df.index.month
    months_unique = df['month'].unique()
    first_month = months_unique[0]
    len_first_month = len(df[df['month']==first_month])
    new_values = np.empty(len_first_month)
    new_values[:] = np.NaN
    new_index = df[df['month']==first_month].index
    df['diff_Sunday'] = np.NaN

    for month in months_unique:
        if month > first_month:
            index_current = df.loc[df['month']==month].index
            index_previous = df.loc[df['month']==month-1].index
            df_current = df.loc[index_current, :].copy()
            df_previous = df.loc[index_previous, :].copy()
            
            df_s_previous = df_previous.loc[df_previous['weekday'] == 6].copy()
            values_s_previous = df_s_previous.groupby(['hour', 'minute']).mean()[feature]
            
            df_nos_previous = df_previous.loc[df_previous['weekday'] != 6].copy()
            values_nos_previous = df_nos_previous.groupby(['hour', 'minute']).mean()[feature]

            values_diff = values_nos_previous - values_s_previous

            current_s_number = int(len(df_current[df_current['weekday']==6]) / 144)
            values_diff_current = list(values_diff) * current_s_number

            df.loc[((df.index.isin(index_current)) & (df['weekday'] == 6)), 'diff_Sunday'] = values_diff_current

            df['diff_Sunday'] = df['diff_Sunday'].fillna(0)

    return df

In [367]:
def generate_feature_previous_month(data, feature):

    df = data.copy()
    df['month'] = df.index.month
    months_unique = df['month'].unique()
    first_month = months_unique[0]
    len_first_month = len(df[df['month']==first_month])
    new_values = np.empty(len_first_month)
    new_values[:] = np.NaN

    for month in months_unique:
        if month > first_month:
            index_current = df.loc[df['month']==month].index
            index_previous = df.loc[df['month']==month-1].index
            len_current = len(df.loc[index_current])
            len_previous = len(df.loc[index_previous])
            values_previous = df.loc[index_previous, feature].copy()
            if len_previous > len_current:
                values_current = values_previous.iloc[:len_current].values.copy()
            else:
                values_current = values_previous.values.copy()
                len_diff = len_current - len_previous
                values_diff = values_current[-len_diff:]
                values_current = np.concatenate((values_current, values_diff), axis=0)
            new_values = np.append(new_values, values_current)

    return new_values

In [18]:
def cv_split_indexes(data, start, train_size, test_size, size_unit, n_splits, freq):

    '''
    Sliding window TS-split
    '''
    
    df = data.copy()

    train_window = {size_unit: train_size}
    train_offset = pd.offsets.DateOffset(**train_window)

    if isinstance(start, str):
        start_date = pd.to_datetime(start) - train_offset
    else:
        start_date = start - train_offset

    train_indexes_list = []
    test_indexes_list = []

    for n in arange(n_splits):
        
        train_window = {size_unit: train_size}
        train_offset = pd.offsets.DateOffset(**train_window)
        train_start = start_date
        train_end = train_start + train_offset
        train_indexes = pd.date_range(train_start, train_end, freq=freq)[:-1]
        train_indexes_list.append(train_indexes)
        
        test_window = {size_unit: test_size}
        test_offset = pd.offsets.DateOffset(**test_window)
        test_start = start_date + train_offset
        test_end = test_start + test_offset
        test_indexes = pd.date_range(test_start, test_end, freq=freq)[:-1]
        test_indexes_list.append(test_indexes)

        start_date = start_date + test_offset

    return train_indexes_list, test_indexes_list

In [ ]:
def cv_model_evaluation(
        data, start, train_size, test_size, size_unit, n_splits, freq,
        orders, fourier_periods, fourier_orders, fourier_orders_type='iter', exog_variables=None):

    time_start = time.time()
    
    train_indexes, test_indexes = cv_split_indexes(
        data, start, train_size, test_size, size_unit, n_splits, freq)

    if len(train_indexes) != len(test_indexes):
        print('ERROR')

    results = {}
    results_ = {}
    models = {}
    models_names = {}
    iteration = 0

    for split_number in arange(len(train_indexes)):
        
        train_data = data.loc[train_indexes[split_number]]
        test_data = data.loc[test_indexes[split_number]]
        
        if exog_variables is not None:
            exog_variables_train = exog_variables.loc[train_indexes[split_number]]
            exog_variables_test = exog_variables.loc[test_indexes[split_number]]

        for order in orders:
            for fourier_period in fourier_periods:
                # if 'iter', than check all combinations of fourier_periods
                # e.g. fourier_periods == [1,2,3,4] than check all posibble combinations (1,1,1,1), (1,1,1,2), (1,1,1,3), etc.
                if fourier_orders_type == 'iter':
                    fourier_orders_for_iteration = itertools.product(fourier_orders, repeat=len(fourier_period))
                # if 'mono', than
                # e.g. fourier_periods == [1,2,3,4] than check [1,1,1,1], [2,2,2,2], [3,3,3,3], [4,4,4,4]
                # for length 'fourier_period' == 4
                elif fourier_orders_type == 'mono':
                    fourier_orders_for_iteration = [[i]*len(fourier_period) for i in fourier_orders]
                else:
                    raise ValueError("'fourier_orders_type' have to be 'iter' or 'mono'")
                for fourier_order in fourier_orders_for_iteration:
                    # create df with fourier exogs for train dataset
                    exogs_df_train = pd.DataFrame()
                    # create df with fourier exogs for test dataset
                    exogs_df_test = pd.DataFrame()
                    for p, k in zip(fourier_period, fourier_order):

                        fourier_train = statsmodels.tsa.deterministic.Fourier(p, k)
                        exogs_train = fourier_train.in_sample(train_data.index)
                        exogs_df_train = exogs_train.join(exogs_df_train)

                        fourier_test = statsmodels.tsa.deterministic.Fourier(p, k)
                        exogs_test = fourier_test.in_sample(test_data.index)
                        exogs_df_test = exogs_test.join(exogs_df_test)

                    if exog_variables is not None:
                        exogs_df_train = exogs_df_train.join(exog_variables_train)
                        exogs_df_test = exogs_df_test.join(exog_variables_test)
                    
                    # fit model
                    model = SARIMAX(
                        train_data, exog=exogs_df_train,
                        order=order,
                        seasonal_order=(0, 0, 0, 0),
                        initialization='approximate_diffuse',
                        # enforce_stationarity=False,
                        # enforce_invertibility=False
                    ).fit(maxiter=1000, disp=False)

                    # calculate steps for forecast
                    steps = len(test_data)
                    # get forecast
                    forecast = model.get_forecast(steps=steps, exog=exogs_df_test)
                    # y_pred
                    y_pred = forecast.predicted_mean
                    # y_test
                    y_test = test_data
                    # RMSE
                    rmse = root_mean_squared_error(y_pred, y_test)

                    k_list = list(fourier_order)

                    model_name = (tuple(order), list(fourier_period), list(fourier_order))
                    model_name_str = f'{tuple(order)}, {list(fourier_period)}, {list(fourier_order)}'

                    models_names[model_name_str] = model_name
                    
                    if model_name_str in results:
                        results[model_name_str] = np.append(results[model_name_str], rmse)
                    else:
                        results[model_name_str] = np.array([rmse])

                    print(
                    f'Iteration {iteration}' '\n' \
                    f' - Split: {split_number};' '\n' \
                    f' - Model: {order};' '\n' \
                    f' - Fourier period: {fourier_period};' '\n' \
                    f' - Fourier order {fourier_order}.')

                    iteration += 1

    # change models in 'result' by 'modelN'-type names and create separate models list
    for (i, key), m in zip(enumerate(results.keys()), models_names.keys()):
        # models[f'model{i}'] = key
        models[f'model{i}'] = models_names[m]
        results_[f'model{i}'] = results[key]

    results_full = {}
    results_full['models'] = models
    results_full['splits'] = results_
    
    clear_output()
    
    time_finish = time.time() - time_start
    time_finish = dt.timedelta(seconds=np.round(time_finish))

    print(f'Execution time: {time_finish}')

    return results_full

In [ ]:
def datasets_w_fourier(
        fourier_period: list,
        fourier_order: list,
        train_data,
        eval_data,
        train_exog=None,
        eval_exog=None):

    # create df with fourier exogs for train dataset
    train_df = pd.DataFrame()
    # create df with fourier exogs for test dataset
    eval_df = pd.DataFrame()
    
    for p, o in zip(fourier_period, fourier_order):

        train_fourier = statsmodels.tsa.deterministic.Fourier(p, o)
        train_fourier_exogs = train_fourier.in_sample(train_data.index)
        train_df = train_fourier_exogs.join(train_df)

        eval_fourier = statsmodels.tsa.deterministic.Fourier(p, o)
        eval_fourier_exogs = eval_fourier.in_sample(eval_data.index)
        eval_df = eval_fourier_exogs.join(eval_df)

    train_df = train_df.join(train_exog)
    eval_df = eval_df.join(eval_exog)

    return train_df, eval_df

In [ ]:
def cv_arima_evaluation_error(
        data_train,
        data_eval,
        order,
        exog_train=None,
        exog_eval=None,
        metric=root_mean_squared_error,
        exec_time=True, clear_output_include=True):

    '''
    Metric: sklearn-like metric with (y_true, y_pred)
    '''

    time_start = time.time()

    model = SARIMAX(
        endog=data_train,
        exog=exog_train,
        order=order,
        seasonal_order=(0, 0, 0, 0)
    ).fit(maxiter=1000, disp=False)

    if clear_output_include:
        clear_output()

    y_true = data_eval
    y_pred = model.get_forecast(steps=len(y_true), exog=exog_eval).predicted_mean

    result = metric(y_true, y_pred)
    
    time_finish = time.time() - time_start
    time_finish = dt.timedelta(seconds=np.round(time_finish))

    if exec_time:
        print(f'Execution time: {time_finish}')

    return result

In [ ]:
def get_peaks_indicies(data, boundary):
    peaks, _ = scipy.signal.find_peaks(data, height=boundary)
    return peaks

In [ ]:
def index_for_occurrence(text, token, occurrence):
    gen = (i for i, l in enumerate(text) if l == token)
    for _ in range(occurrence - 1):
        next(gen)
    return next(gen)

In [ ]:
def plot_forecast(
        y_true,
        y_pred,
        forecast,
        slice_series=[None, None],
        ci=[80, 95],
        palette=None,
        alpha_actual=0.75,
        alpha_forecast=0.75,
        alphas=[0.25, 0.15],
        legend=True,
        display_intervals=True,
        ax=None):

    if ax is None: ax = plt.gca()

    palette = palette or plt.rcParams['axes.prop_cycle'].by_key()['color']
    ci_alphas = [(100 - i)*0.01 for i in ci]
    slice_series_ = slice(slice_series[0], slice_series[1])

    y_true = y_true[slice_series_]
    y_pred = y_pred[slice_series_]
    
    ax.plot(y_true, color=palette[0], alpha=alpha_actual, label='Actual')
    ax.plot(y_pred, color=palette[1], alpha=alpha_forecast, label='Forecast')

    if display_intervals:

        level0 = forecast.conf_int(alpha=ci_alphas[0])
        level1 = forecast.conf_int(alpha=ci_alphas[1])
    
        label0 = f'Level {ci[0]}%'
        label1 = f'Level {ci[1]}%'

        level0 = level0[slice_series_]
        level1 = level1[slice_series_]

        ci_levels = [level0, level1]
        ci_labels = [label0, label1]
        ci_palette = [palette[2], palette[2]]

        for le, la, c, a in zip(ci_levels, ci_labels, ci_palette, alphas):
        
            ax.fill_between(
                x=y_pred.index,
                y1=le.iloc[:, 0],
                y2=le.iloc[:, 1],
                lw=0,
                color=c,
                alpha=a,
                label=la)

    if legend:
        ax.legend(
            **legend_inline(),
            **legend_create_handles(
                4, kind=['l', 'l', 'r', 'r'],
                labels=['Actual', 'Forecast', label0, label1],
                colors=[palette[0], palette[1], palette[2], palette[2]],
                alphas=[alpha_actual, alpha_forecast, alphas[0], alphas[1]]))